#  Welcome to the Cyclistic bike-share analysis case study! 
In this hypothetical scenario, I am as ajunior data analyst  tasked with developing strategies to convert casual riders into annual members for a fictional bike-share program in Chicago. I will follow the data analysis process—Ask, Prepare, Process, Analyze, Share, and Act—to analyze historical bike trip data, identify usage patterns, and provide actionable recommendations. My insights and visualizations will support targeted marketing strategies and inform recommendations for the executive team.

>  # Ask

The **business task** is to analyze historical bike trip data to understand the usage differences between casual riders and annual members, and to develop actionable marketing strategies that will convert casual riders into annual members.

>  # Prepare

The analysis utilizes Cyclistic’s historical trip records, which are available through [the Cyclistic data repository](https://divvy-tripdata.s3.amazonaws.com/index.html) and provided by Motivate International Inc. under this [license](http://https://www.divvybikes.com/data-license-agreement). 
For this case study, I will use the data from the previous 12 months, spanning from August 2023 to July 2024, which is provided as separate files.

These files include the following columns:

* ride_id: A unique identifier for each bike ride.
* started_at: The date and time when the ride began.
* ended_at: The date and time when the ride ended.
* start_station_name: The name of the station where the ride started.
* end_station_name: The name of the station where the ride ended.
* member_casual: Indicates whether the rider is an annual member or a casual rider.

Additional columns in the datasets may include various details about the ride, but these are the primary fields relevant to the analysis.

I will combine these files to analyze and identify trends in bike usage patterns.

In [ ]:
# Load necessary libraries
library(data.table)
library(lubridate)
library(dplyr)
library(knitr)
library(ggplot2)

**Combine CSV files**

Here, I will combine the previous 12 months csv files into a single dataset.

In [ ]:
# Define the base directory where CSV files are stored
base_directory <- "/kaggle/input/cyclistic-data"

In [ ]:
# The CSV files for previous 12 months 8-2023 to 7-2024
months_years <- c("202308","202309", "202310", "202311", "202312", 
                  "202401", "202402", "202403", "202404", 
                  "202405", "202406", "202407")

In [ ]:
# Paths to each CSV file
csv_files <- sapply(months_years, function(month_year) {
  file.path(base_directory, paste0(month_year, "-divvy-tripdata"), paste0(month_year, "-divvy-tripdata.csv"))
})

# Read each CSV file
read_csv <- function(file_path) {
  if (file.exists(file_path)) {
    fread(file_path)
  } else {
    warning(paste("File does not exist:", file_path))
    return(NULL)
  }
}

In [ ]:
# Read and combine all CSV files
all_data <- lapply(csv_files, read_csv)
combined_data <- rbindlist(all_data, use.names = TRUE)#, fill = TRUE)

In [ ]:
head(combined_data)

In [ ]:
# Check the number of rows in the combined data
num_rows <- nrow(combined_data)
print(num_rows)

In [ ]:
# Columns info
str(combined_data)

**Now, The data is prepared and ready to clean,manipulate...etc**

>  # Process

Here, I will process the data by handling missing values, checking for duplicate rows, and performing feature engineering, which involves adding new columns extracted from existing ones.

**Handling Missing Values**

In [ ]:
# Count the number of rows contain null values
rows_with_na <- sum(rowSums(is.na(combined_data)) > 0)
print(paste("Number of rows with NA value:", rows_with_na))

In [ ]:
# Count the number of missing values per column
na_count_per_column <- colSums(is.na(combined_data))
# Create a bar plot of missing values by column
barplot(na_count_per_column,
        main = "Number of Missing Values by Column",
        xlab = "",
        ylab = "Number of Missing Values",
        las = 2,                    
        col = "skyblue",            
        border = "white")           


In [ ]:
# Remove rows with any NA values
df_cleaned <- na.omit(combined_data)

# Check the number of rows after removal
print(paste("Number of rows after removing rows with NA values:", nrow(df_cleaned)))

In [ ]:
# Make sure no null values
na_count_per_column <- colSums(is.na(df_cleaned))

# Create a bar plot of missing values by column
barplot(na_count_per_column,
        main = "Number of Missing Values by Column",
        xlab = "",
        ylab = "Number of Missing Values",
        las = 2,                    # Rotate x-axis labels for better readability
        col = "skyblue",            # Bar color
        cex.names = 0.7,            # Adjust label size
        border = "white")           # No border color


**Check Duplicates**

In [ ]:
# Count the number of duplicate rows
duplicate_rows <- sum(duplicated(df_cleaned))
print(paste("Number of duplicate rows:", duplicate_rows))

 **Feature engineering**
 
 Adding ride_length_seconds Column by find the diff between started_at and ended_at columns

In [ ]:
# Ensure df_cleaned is a data.table
df_cleaned <- as.data.table(df_cleaned)

# Calculate the duration in seconds
df_cleaned[, ride_length_seconds := as.numeric(difftime(ended_at, started_at, units = "secs"))]


In [ ]:
str(df_cleaned$ride_length_seconds)

In [ ]:
head(df_cleaned)

 **Feature engineering**
 
Adding day_of_week column that calculate the day of the week that each ride
started 
* noting that 1 = Sunday and 7 = Saturday

In [ ]:
df_cleaned[, day_of_week := wday(started_at, week_start = 7)]  # week_start = 7 means Sunday starts the week


In [ ]:
head(df_cleaned[, .(started_at, day_of_week)])

In [ ]:
head(df_cleaned)

**Handling negative values in ride_length_seconds**

In [ ]:
# Check negative values in ride_length_seconds
negative_values <- df_cleaned[ride_length_seconds < 0]
print(negative_values)


In [ ]:
# Setting negative values to zero
df_cleaned[ride_length_seconds < 0, ride_length_seconds := 0]  

We Have some empty cells (Not null but empty)in start_station_name and end_station_name so we will create df_filtered data to use it when analyze these 2 columns

In [ ]:
empty_start_station_count <- sum(df_cleaned$start_station_name == "")
empty_end_station_count <- sum(df_cleaned$end_station_name == "")
print(paste("Empty cells in 'start_station_name':", empty_start_station_count))
print(paste("Empty cells in 'end_station_name':", empty_end_station_count))

In [ ]:
df_filtered <- df_cleaned %>%
  filter(start_station_name != "" & end_station_name != "")


**Now we processed the data, ready to Analyze!**

>  # Analyze

Here, we will do calculations and statistical analytics

In [ ]:
# Number of rides based on user type
count_by_user_type <- df_cleaned %>%
  group_by(member_casual) %>%
  summarise(count = n())

# Print the counts as table
kable(count_by_user_type,caption = "Number of rides based on user type")

**Descriptive statistics on ride_length_seconds**

In [ ]:
mean_duration <- mean(df_cleaned$ride_length_seconds, na.rm = TRUE)
median_duration <- median(df_cleaned$ride_length_seconds, na.rm = TRUE)
max_duration <- max(df_cleaned$ride_length_seconds, na.rm = TRUE)
min_duration <- min(df_cleaned$ride_length_seconds, na.rm = TRUE)

# Print results
cat("Mean Duration in Seconds:", mean_duration, "\n")
cat("Median Duration in Seconds:", median_duration, "\n")
cat("Max Duration in Seconds:", max_duration, "\n")
cat("Min Duration in Seconds:", min_duration, "\n")

**Compare member_casual by mean, median, max, min**

In [ ]:
mean_by_user_type <- df_cleaned[, .(mean_duration = mean(ride_length_seconds, na.rm = TRUE)), by = member_casual]
median_by_user_type <- df_cleaned[, .(median_duration = median(ride_length_seconds, na.rm = TRUE)), by = member_casual]
max_by_user_type <- df_cleaned[, .(max_duration = max(ride_length_seconds, na.rm = TRUE)), by = member_casual]
min_by_user_type <- df_cleaned[, .(min_duration = min(ride_length_seconds, na.rm = TRUE)), by = member_casual]

# Combine the statistics into one data table
combined_stats <- merge(mean_by_user_type, median_by_user_type, by = "member_casual")
combined_stats <- merge(combined_stats, max_by_user_type, by = "member_casual")
combined_stats <- merge(combined_stats, min_by_user_type, by = "member_casual")


print(combined_stats)



**Average ride duration in seconds by the day**

In [ ]:
avg_ride_time_by_day <- aggregate(df_cleaned$ride_length_seconds ~ df_cleaned$member_casual + df_cleaned$day_of_week, 
                                  FUN = mean)
# Rename columns for clarity
colnames(avg_ride_time_by_day) <- c("member_casual", "day_of_week", "average_ride_time")

# Print the results as a formatted table
kable(avg_ride_time_by_day, caption = "Average Ride Time by Day of Week and User Type")

**Rideable Type**

In [ ]:
# Count the number of rides by member_casual and rideable_type
ride_counts <- df_cleaned %>%
  group_by(member_casual, rideable_type) %>%
  summarise(number_of_rides = n(), .groups = 'drop')

# Print the results as a formatted table
kable(ride_counts, caption = "Number of Rides by User Type and Rideable Type")

**Start & End Stations**

In [ ]:
# Count the number of rides by start station and user type
start_station_counts <- df_filtered %>%
  group_by(member_casual, start_station_name) %>%
  summarise(start_count = n(), .groups = 'drop') %>%
  arrange(member_casual, desc(start_count))

# View the top start stations for each user type
head(start_station_counts)

end_station_counts <- df_filtered %>%
  group_by(member_casual, end_station_name) %>%
  summarise(end_count = n(), .groups = 'drop') %>%
  arrange(member_casual, desc(end_count))

# View the top end stations for each user type
head(end_station_counts)

>  # Share


Here, I will visualize the analysis to reveal findings on how annual members and casual riders use Cyclistic bikes differently.

In [ ]:
# Plot count_by_user_type
ggplot(count_by_user_type, aes(x = member_casual, y = count, fill = member_casual)) +
  geom_bar(stat = "identity") +
  labs(title = "Number of Rides Based on User Type", x = "User Type", y = "Number of Rides") +
  theme_minimal() +
  scale_fill_brewer(palette = "Set1")

# Plot mean duration 
ggplot(mean_by_user_type, aes(x = member_casual, y = mean_duration, fill = member_casual)) +
  geom_bar(stat = "identity") +
  labs(title = "Mean Ride Duration by User Type",
       x = "User Type",
       y = "Mean Duration (seconds)") +
  theme_minimal() +
  scale_fill_brewer(palette = "Set1") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))


> * **Member Users**: Higher number of rides indicates regular use.
> * **Casual Users**: Higher average ride duration suggests that they may be using the service for longer trips.

In [ ]:
# Plot Average duration by day 
# Note: 1 = Sunday, 7 = Saturday
ggplot(avg_ride_time_by_day, aes(x = day_of_week, y = average_ride_time, color = member_casual, group = member_casual)) +
  geom_line() +
  geom_point() +
  labs(title = "Average Ride Duration by Day of the Week and Member Type",
       x = "Day of the Week",
       y = "Average Ride Duration (seconds)") +
  theme_minimal() +
  scale_color_brewer(palette = "Set1")


> * **Member Users**: Shorter, more consistent ride times across all days.
> * **Casual Users**: Longer ride durations throughout the week, with peaks on weekends.

In [ ]:
# Plot the rideable type
ggplot(ride_counts, aes(x = rideable_type, y = number_of_rides, fill = member_casual)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Number of Rides by User Type and Rideable Type",
       x = "Rideable Type",
       y = "Number of Rides") +
  theme_minimal() +
  scale_fill_brewer(palette = "Set1") 

> * **Member Users**: Primarily use classic bikes, but also have a high number of rides on electric bikes.
> * **Casual Users**: Most commonly use electric bikes and classic bikes, with significantly fewer rides on docked bikes.

In [ ]:
# Plot Top station for casual users 
# start stations
ggplot(start_station_counts %>% filter(member_casual == "casual") %>% head(5), aes(x = reorder(start_station_name, -start_count), y = start_count)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  labs(title = "Top 5 Start Stations for Casual Riders", x = "Start Station", y = "Number of Rides") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

# end stations
ggplot(end_station_counts %>% filter(member_casual == "casual") %>% head(5), aes(x = reorder(end_station_name, -end_count), y = end_count)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  labs(title = "Top 5 End Stations for Casual Riders", x = "End Station", y = "Number of Rides") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

> * The visual shows that Streeter Dr & Grand Ave is the most popular **start station** among casual riders, followed by DuSable Lake Shore Dr & Monroe St. This indicates high traffic and potential high demand in these areas.
> * Similarly, Streeter Dr & Grand Ave also ranks as the top **end station**, suggesting it is a major destination. The other top end stations reflect popular travel routes or destinations. 

In [ ]:
# Plot Top station for member users 
# start stations
ggplot(start_station_counts %>% filter(member_casual == "member") %>% head(5), aes(x = reorder(start_station_name, -start_count), y = start_count)) +
  geom_bar(stat = "identity", fill = "coral") +
  labs(title = "Top 5 Start Stations for Member Riders", x = "Start Station", y = "Number of Rides") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

# end stations
ggplot(end_station_counts %>% filter(member_casual == "member") %>% head(5), aes(x = reorder(end_station_name, -end_count), y = end_count)) +
  geom_bar(stat = "identity", fill = "coral") +
  labs(title = "Top 5 End Stations for Member Riders", x = "End Station", y = "Number of Rides") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))




> * The visual indicates that Clinton St & Washington Blvd is the most popular **start station** among members, followed by Kingsbury St & Kinzie St. 
> * Clinton St & Washington Blvd also ranks as the top **end station**, indicating it is a key destination. 

>  # Act

**Recommendations** based on the analysis

* **Target Weekend Promotions:**
Offer special weekend deals for casual riders, emphasizing the benefits of membership for longer rides and additional perks during peak times.

* **Enhance Services at Key Stations:**
Improve bike availability and services at top stations like Streeter Dr & Grand Ave and Clinton St & Washington Blvd to enhance the membership experience.

* **Advertise Membership Benefits at Top Stations:**
Promote membership benefits prominently at popular stations for casual users to increase visibility and attract potential members. Highlight advantages like cost savings and exclusive perks.

* **Introduce Tiered Membership Packages:**
Create membership packages tailored to different rideable types, such as a premium package for electric bikes and a standard package for classic bikes. This customization allows casual riders to choose a membership that suits their preferences.